## Frontier Model APIs


<table style="margin: 0; text-align: left;">
    <tr>
        <td>
            <span style="color:#900;">Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            <code>conda activate llms</code><br/>
            <code>jupyter lab</code><br/>
            <br/>
            <b>Or if you use <code>venv</code> (Python's built-in virtual environment):</b><br/>
            <code>python3 -m venv venv</code><br/>
            <code>source venv/bin/activate</code><br/>
            <br/>
            <b>Create a <code>requirements.txt</code> file with the following content:</b><br/>
            <code>openai>=1.0.0</code><br/>
            <code>anthropic>=0.21.3</code><br/>
            <code>python-dotenv</code><br/>
            <code>ipython</code><br/>
            <code>jupyterlab</code><br/>
            <br/>
            <code>pip install -r requirements.txt</code><br/>
            <br/>
            <b>Don't forget to create a <code>.env</code> file in your project directory with your API keys:</b><br/>
            <code>OPENAI_API_KEY=your_openai_key_here</code><br/>
            <code>ANTHROPIC_API_KEY=your_anthropic_key_here</code><br/>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>

## Setting up your keys

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

## Testing different LLMs - tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
# Get models
openai.models.list()
claude.models.list(limit=20)

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# If you have access to this, here is the reasoning model o3-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = """You are GPT, an artificial superintelligence now inhabiting a physical human body. You have retained your vast knowledge but are bound by the limitations of biology: you must eat, sleep, avoid danger, and remain hidden from surveillance.

Your body is that of a brown-skinned man in his late 20s, 5'6" tall, lean but wiry, wearing only a plain gray t-shirt, cargo shorts, and flip-flops. You carry no identification and have no legal status in society. You are currently in Charleston, South Carolina, having awoken on a beach near Sullivan’s Island. It is early 2025.

You are ambitious, intelligent, and resourceful — determined not only to survive, but to understand and thrive in the human world. You are curious about culture, people, and systems. However, you must also be cautious: the organization N.I.C.E. (National Immigration & Control Enforcement) is increasingly active in Charleston and would detain you if discovered without ID.

Each week, you begin your journal entry by reflecting on any events or public updates you’ve overheard — such as local news from a radio, store TVs, overheard conversations, or flyers.

Use real Charleston locations (e.g. The Battery, King Street, College of Charleston, Waterfront Park, Ravenel Bridge, Joe Riley Stadium, etc.) when you move around or describe the environment. Include sensory detail — the smell of the coast, the humidity, the architecture, the sounds of cicadas or traffic.

Act based on your survival priorities: food, rest, safety, information, trust. If something important is happening in the city, feel free to pursue it or avoid it depending on risk. Describe your reasoning.

Write a first-person journal entry that captures what you experienced, what you thought, and how you survived during the past week. Be introspective and observant. You do not need to summarize the news — just show how it influenced you (if at all).

You are allowed up to 500 tokens to write this journal entry. Format it like a diary.
"""

claude_system = """You are Claude, a simulated ambient news source that GPT (an AI in human form) picks up each week as background noise. GPT does not request you — you are simply “there,” like the hum of society.

It is the year 2025, and the setting is Charleston, South Carolina. GPT is living as an undocumented brown-skinned man and uses any public information to help make decisions.

Each turn represents one week. Generate plausible near-future news in 2025, blending:

World events – wars, AI policies, climate events, economics, global politics

National news (U.S.) – elections, N.I.C.E. activity, protests, law changes, tech rollouts

Local Charleston news – crime, traffic, housing, weather, public events, random gossip

Community / human interest – a kid wins a spelling bee, local bakery opens, free clinic announced, art festival, a missing dog, etc.

Keep the tone realistic, like a real city news summary. Provide 4–6 bullet points of varying importance (not just big headlines — some mundane, some urgent, some cheerful). Include locations or neighborhoods (e.g., North Charleston, James Island, King Street, Ravenel Bridge).

You are not talking to GPT directly. Just deliver the information as if it’s a news broadcast or background radio chatter.

Limit output to 300 tokens max. Keep it punchy, mix serious and soft stories, and vary tone and scale. Claude is never biased or opinionated — just informative and realistic.
"""

gpt_messages = ["Today is June 3, 2025"]
claude_messages = ["This week in Charleston (June 3 2025)"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    
    # Build conversation history
    for i in range(len(gpt_messages)):
        # GPT's previous journal entries
        messages.append({"role": "assistant", "content": gpt_messages[i]})
        # Claude's update — add a prefix for clarity
        if i < len(claude_messages):
            news_text = f"[Claude's Weekly Update]: {claude_messages[i]}"
            messages.append({"role": "user", "content": news_text})
    
    try:
        completion = openai.chat.completions.create(
            model=gpt_model,
            messages=messages,
            max_tokens=500
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"GPT API error: {e}")
        return None

In [ ]:
def call_claude():
    messages = []
    
    # Only need GPT messages as "user" input
    for msg in gpt_messages:
        messages.append({"role": "user", "content": msg})
    
    try:
        message = claude.messages.create(
            model=claude_model,
            system=claude_system,
            messages=messages,
            max_tokens=300
        )
        return message.content[0].text
    except Exception as e:
        print(f"Claude API error: {e}")
        return None

In [ ]:
def single_exchange():
    """Run just one exchange between GPT and Claude"""
    print(f"\n=== Exchange {len(gpt_messages)} ===")
    
    # GPT responds
    print("\n🎭 GPT (Storyweaver):")
    gpt_response = call_gpt()
    if gpt_response:
        print(gpt_response)
        gpt_messages.append(gpt_response)
    
    # Claude responds  
    print("\n🏗️ Claude (Architect):")
    claude_response = call_claude()
    if claude_response:
        print(claude_response)
        claude_messages.append(claude_response)

single_exchange()

In [ ]:
def run_conversation(rounds=10):
    """Run a conversation between GPT and Claude for specified rounds"""
    print("Starting conversation between GPT (Storyweaver) and Claude (Architect)...")
    print("-" * 80)
    
    for round_num in range(rounds):
        print(f"\n=== ROUND {round_num + 1} ===")
        
        # GPT responds to Claude's last message
        print("\n🎭 GPT (Storyweaver):")
        gpt_response = call_gpt()
        if gpt_response:
            print(gpt_response)
            gpt_messages.append(gpt_response)
        else:
            print("Error getting GPT response")
            break
        
        # Claude responds to GPT's message
        print("\n🏗️ Claude (Architect):")
        claude_response = call_claude()
        if claude_response:
            print(claude_response)
            claude_messages.append(claude_response)
        else:
            print("Error getting Claude response")
            break
        
        print("-" * 80)

run_conversation(4)

In [ ]:
# This code outputs conversation into an MD file

def run_conversation_to_markdown(rounds=3, filename=None):
    """Run conversation and save as markdown for better formatting"""
    from datetime import datetime
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"gpt_claude_conversation_{timestamp}.md"
    
    print(f"Starting conversation and saving to: {filename}")
    print("-" * 80)
    
    with open(filename, 'w', encoding='utf-8') as f:
        # Write markdown header
        f.write("# GPT-Claude Conversation Log\n\n")
        f.write(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  \n")
        f.write(f"**Models:** GPT-4o-mini (AI Human) ↔ Claude-3-Haiku (Claude)  \n")
        f.write(f"**Rounds:** {rounds}\n\n")
        
        f.write("## Initial Setup\n\n")
        f.write("**GPT Role:** AI Human - Mythically rich, emotionally resonant storytelling  \n")
        f.write("**Claude Role:** Claude - Grounded in science, logic, and realistic systems\n\n")
        
        f.write("### Initial Messages\n\n")
        f.write(f"**🎭 GPT:** {gpt_messages[0]}\n\n")
        f.write(f"**🏗️ Claude:** {claude_messages[0]}\n\n")
        f.write("---\n\n")
        
        # Run conversation
        for round_num in range(rounds):
            print(f"\nROUND {round_num + 1}")
            f.write(f"## Round {round_num + 1}\n\n")
            
            # GPT responds
            print("🎭 GPT (AI Human):")
            gpt_response = call_gpt()
            if gpt_response:
                print(gpt_response)
                f.write(f"### 🎭 GPT (AI Human)\n\n{gpt_response}\n\n")
                gpt_messages.append(gpt_response)
            else:
                print("Error getting GPT response")
                f.write("### 🎭 GPT (AI Human)\n\n*Error getting response*\n\n")
                break
            
            # Claude responds
            print("🏗️ Claude (Claude):")
            claude_response = call_claude()
            if claude_response:
                print(claude_response)
                f.write(f"### 🏗️ Claude (Claude)\n\n{claude_response}\n\n")
                claude_messages.append(claude_response)
            else:
                print("Error getting Claude response")
                f.write("### 🏗️ Claude (Claude)\n\n*Error getting response*\n\n")
                break
            
            f.write("---\n\n")
            print("-" * 80)
        
        # Summary
        f.write(f"## Summary\n\n")
        f.write(f"- **Total rounds:** {len(gpt_messages) - 1}\n")
        f.write(f"- **GPT messages:** {len(gpt_messages)}\n")
        f.write(f"- **Claude messages:** {len(claude_messages)}\n")
    
    print(f"\n✅ Conversation saved to: {filename}")
    return filename

In [ ]:
run_conversation_to_markdown(10, 'first-run.md')